# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 09

Group Members:
- Ananda Farhan Raihandra (18222084)
- Alfaza Naufal Zakiy (18222126)

In [ ]:
%pip install imbalanced-learn

In [ ]:
%pip install tqdm

## Import Libraries

In [7]:
# Pustaka untuk manipulasi data
import pandas as pd
import numpy as np

# Pustaka untuk data splitting dan evaluasi
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Implementasi algoritma scikit-learn
from sklearn.neighbors import KNeighborsClassifier  # KNN dari scikit-learn
from sklearn.naive_bayes import GaussianNB  # Gaussian Naive Bayes dari scikit-learn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter
from tqdm import tqdm


## Import Dataset

In [3]:
# Load Dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submission_template = pd.read_csv('sample_submission.csv')

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [4]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...

# Pilih fitur (X) dan target (y)
LABEL = 'label'  # Nama kolom target
X = train_data.drop(columns=[LABEL])  # Data fitur
y = train_data[LABEL]  # Data target

# Split data menjadi data latih dan validasi
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data latih:", X_train.shape)
print("Data validasi:", X_val.shape)


Data latih: (112323, 55)
Data validasi: (28081, 55)


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [5]:
# Write your code here


# Membuat objek SimpleImputer untuk imputasi data
numerical_imputer = SimpleImputer(strategy='mean')  # Untuk nilai numerik, gunakan mean
categorical_imputer = SimpleImputer(strategy='most_frequent')  # Untuk nilai kategori, gunakan nilai yang paling sering muncul

# Identifikasi kolom numerik dan kategori
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Imputasi data latih
X_train[numerical_cols] = numerical_imputer.fit_transform(X_train[numerical_cols])
X_train[categorical_cols] = categorical_imputer.fit_transform(X_train[categorical_cols])

# Imputasi data validasi
X_val[numerical_cols] = numerical_imputer.transform(X_val[numerical_cols])
X_val[categorical_cols] = categorical_imputer.transform(X_val[categorical_cols])

# Mengecek apakah ada missing value yang tersisa
print("Missing values in training set:", X_train.isnull().sum().sum())
print("Missing values in validation set:", X_val.isnull().sum().sum())


Missing values in training set: 0
Missing values in validation set: 0


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [463]:
# Write your code here
# Menangani Outlier dengan Clipping
# Menentukan batas bawah dan atas berdasarkan persentil
lower_bound = 0.01  # Batas bawah 1%
upper_bound = 0.99  # Batas atas 99%

# Identifikasi kolom numerik
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Melakukan clipping pada data latih
for col in numerical_cols:
    low = X_train[col].quantile(lower_bound)
    high = X_train[col].quantile(upper_bound)
    X_train[col] = X_train[col].clip(lower=low, upper=high)

# Melakukan clipping pada data validasi
for col in numerical_cols:
    low = X_val[col].quantile(lower_bound)
    high = X_val[col].quantile(upper_bound)
    X_val[col] = X_val[col].clip(lower=low, upper=high)

# Mengecek apakah ada outlier yang tersisa
print("Outlier handling selesai.")


Outlier handling selesai.


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [464]:
# Write your code here
# Menghapus duplikat dari data latih
initial_train_shape = X_train.shape  # Menyimpan bentuk awal data
X_train = X_train.drop_duplicates(keep='first')
y_train = y_train.loc[X_train.index]  # Menyesuaikan label dengan data fitur

# Menghapus duplikat dari data validasi
initial_val_shape = X_val.shape
X_val = X_val.drop_duplicates(keep='first')
y_val = y_val.loc[X_val.index]  # Menyesuaikan label dengan data fitur

# Menampilkan hasil penghapusan duplikat
print(f"Data latih sebelum: {initial_train_shape}, setelah: {X_train.shape}")
print(f"Data validasi sebelum: {initial_val_shape}, setelah: {X_val.shape}")


Data latih sebelum: (112323, 55), setelah: (112323, 55)
Data validasi sebelum: (28081, 55), setelah: (28081, 55)


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [465]:
# Handling Missing Values untuk URL
X_train['URL'] = X_train['URL'].fillna('')  # Mengisi NaN dengan string kosong
X_val['URL'] = X_val['URL'].fillna('')      # Mengisi NaN dengan string kosong

# Menambahkan Fitur Baru Berdasarkan URL
X_train['URL_Length'] = X_train['URL'].apply(lambda x: len(str(x)))
X_train['Digit_Count'] = X_train['URL'].apply(lambda x: sum(c.isdigit() for c in str(x)))
X_train['Special_Char_Count'] = X_train['URL'].apply(lambda x: sum(1 for c in str(x) if c in ['@', '#', '/', '?', '=', '&']))
X_val['URL_Length'] = X_val['URL'].apply(lambda x: len(str(x)))
X_val['Digit_Count'] = X_val['URL'].apply(lambda x: sum(c.isdigit() for c in str(x)))
X_val['Special_Char_Count'] = X_val['URL'].apply(lambda x: sum(1 for c in str(x) if c in ['@', '#', '/', '?', '=', '&']))

# Menambahkan Statistik pada Kolom Numerik
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    X_train[f'{col}_log'] = X_train[col].apply(lambda x: np.log(x + 1) if x > 0 else 0)
    X_val[f'{col}_log'] = X_val[col].apply(lambda x: np.log(x + 1) if x > 0 else 0)

# Encoding Kolom Kategori (jika ada)
categorical_cols = X_train.select_dtypes(include=['object']).columns
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category').cat.codes
    X_val[col] = X_val[col].astype('category').cat.codes

# Verifikasi Perubahan pada Data
print("Feature engineering selesai. Dimensi data latih:", X_train.shape)


Feature engineering selesai. Dimensi data latih: (112323, 111)


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [466]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [467]:
# Write your code here


# Identifikasi kolom numerik
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Inisialisasi scaler
scaler = MinMaxScaler()

# Scaling pada data latih
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

# Scaling pada data validasi
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])

# Verifikasi hasil scaling
print("Feature scaling selesai. Contoh data latih setelah scaling:")
print(X_train[numerical_cols].head())


Feature scaling selesai. Contoh data latih setelah scaling:
             id  URLLength  DomainLength  IsDomainIP  CharContinuationRate  \
28021  0.192775   0.312805      0.083333         0.0              0.314286   
71026  0.504182   0.225806      0.291667         1.0              0.859813   
89927  0.642471   0.225806      0.291667         0.0              0.688312   
94654  0.676946   0.290323      0.375000         0.0              1.000000   
35275  0.245080   0.312805      0.250000         0.0              0.859813   

       TLDLegitimateProb  URLCharProb  TLDLength  NoOfSubDomain  \
28021           1.000000     0.620866   0.364555       0.000000   
71026           0.531443     0.672369   0.500000       0.000000   
89927           0.054347     0.620866   0.364555       0.500000   
94654           0.531443     0.809742   0.500000       0.080383   
35275           0.531443     0.620866   0.500000       0.080383   

       HasObfuscation  ...  HasCopyrightInfo_log  NoOfImage_log  NoO

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [468]:
# Write your code here


# Identifikasi kolom kategori
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Inisialisasi LabelEncoder
label_encoders = {}

# Lakukan Label Encoding pada data latih dan validasi
for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))  # Encode data latih
    X_val[col] = le.transform(X_val[col].astype(str))  # Encode data validasi
    label_encoders[col] = le  # Simpan encoder untuk tiap kolom

# Verifikasi hasil encoding
print("Feature encoding selesai. Contoh data latih setelah encoding:")
print(X_train[categorical_cols].head())


Feature encoding selesai. Contoh data latih setelah encoding:
Empty DataFrame
Columns: []
Index: [28021, 71026, 89927, 94654, 35275]


In [469]:
print(X_train.head())
print(X_val.head())


             id  FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP  \
28021  0.192775         0  76839   0.312805    1086      0.083333         0.0   
71026  0.504182         0    631   0.225806    1086      0.291667         1.0   
89927  0.642471      3350  10979   0.225806    1086      0.291667         0.0   
94654  0.676946     23785  45028   0.290323   31571      0.375000         0.0   
35275  0.245080         0  58069   0.312805    1086      0.250000         0.0   

       TLD  CharContinuationRate  TLDLegitimateProb  ...  \
28021  119              0.314286           1.000000  ...   
71026  119              0.859813           0.531443  ...   
89927  119              0.688312           0.054347  ...   
94654  119              1.000000           0.531443  ...   
35275  323              0.859813           0.531443  ...   

       HasCopyrightInfo_log  NoOfImage_log  NoOfCSS_log  NoOfJS_log  \
28021              0.810097       0.625150     0.659062    0.689763   
71026     

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [471]:
# Write your code here




# Sebelum Resampling
print("Distribusi kelas sebelum resampling:", Counter(y_train))

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Terapkan SMOTE ke data latih
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Setelah Resampling
print("Distribusi kelas setelah resampling:", Counter(y_train_resampled))


Distribusi kelas sebelum resampling: Counter({1: 103921, 0: 8402})
Distribusi kelas setelah resampling: Counter({1: 103921, 0: 103921})


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [472]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [473]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [474]:
# Maaf kak, untuk bagian ini tidak saya kerjakan karena sebelumnya sudah saya coba 
# tapi error terus, sehingga saya jalankan manual tanpa menggunakan pipeline

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [479]:
# Kelas KNN from scratch
class KNN:
    def __init__(self, k=3, metric='euclidean'):
        self.k = k
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        elif self.metric == 'minkowski':
            p = 3
            return np.sum(np.abs(x1 - x2) ** p) ** (1 / p)

    def predict(self, X):
        predictions = []
        for x in tqdm(X[:10000], desc="KNN Prediction Progress"):  # Subset data
            distances = np.linalg.norm(self.X_train - x, axis=1)  # Optimasi jarak
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            predictions.append(max(set(k_nearest_labels), key=k_nearest_labels.count))
        return np.array(predictions)

# Data latih dan validasi
# Pastikan `X_train_resampled` dan `y_train_resampled` sesuai dengan hasil preprocessing di kodesementara.py
# Misalnya, gunakan subset untuk percepatan
X_train_subset = X_train_resampled.to_numpy()[:40000]
y_train_subset = y_train_resampled.to_numpy()[:40000]
X_val_subset = X_val.to_numpy()[:10000]
y_val_subset = y_val.to_numpy()[:10000]

# KNN from scratch
knn_scratch = KNN(k=5, metric='euclidean')
knn_scratch.fit(X_train_subset, y_train_subset)
y_pred_scratch = knn_scratch.predict(X_val_subset)

# KNN scikit-learn
knn_sklearn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_sklearn.fit(X_train_subset, y_train_subset)
y_pred_sklearn = knn_sklearn.predict(X_val_subset)

# Evaluasi akurasi
accuracy_scratch = accuracy_score(y_val_subset, y_pred_scratch)
accuracy_sklearn = accuracy_score(y_val_subset, y_pred_sklearn)

# Cetak hasil
print(f"Akurasi KNN from scratch (subset): {accuracy_scratch:.5f}")
print(f"Akurasi KNN scikit-learn (subset): {accuracy_sklearn:.5f}")


KNN Prediction Progress: 100%|██████████| 10000/10000 [07:03<00:00, 23.62it/s]


Akurasi KNN from scratch (subset): 0.82130
Akurasi KNN scikit-learn (subset): 0.82130


## B. Naive Bayes

In [481]:
# Type your code here


class GaussianNaiveBayes:
    def __init__(self):
        self.classes = None
        self.mean = {}
        self.var = {}
        self.priors = {}

    def fit(self, X, y):
        """
        Melatih Gaussian Naive Bayes dengan menghitung rata-rata, variansi,
        dan probabilitas prior untuk setiap kelas.
        """
        self.classes = np.unique(y)
        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = X_c.mean(axis=0)
            self.var[c] = X_c.var(axis=0)
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def _gaussian_density(self, class_idx, x):
        """
        Menghitung densitas Gaussian untuk fitur tertentu dengan regularisasi.
        """
        mean = self.mean[class_idx]
        var = self.var[class_idx] + 1e-9  # Regularisasi variansi
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return np.maximum(numerator / denominator, 1e-9)  # Hindari nol

    def predict(self, X):
        """
        Melakukan prediksi untuk data input.
        """
        y_pred = []
        for x in X:
            posteriors = []
            for c in self.classes:
                prior = np.log(self.priors[c])  # Log prior
                class_conditional = np.sum(np.log(self._gaussian_density(c, x)))
                posterior = prior + class_conditional
                posteriors.append(posterior)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return np.array(y_pred)

# Data latih dan validasi (gunakan hasil preprocessing dari `kodesementara.py`)
X_train_resampled, y_train_resampled  # Data hasil SMOTE
X_val, y_val  # Data validasi

# Gaussian Naive-Bayes from scratch
gnb_scratch = GaussianNaiveBayes()
gnb_scratch.fit(X_train_resampled.to_numpy(), y_train_resampled.to_numpy())
y_pred_scratch = gnb_scratch.predict(X_val.to_numpy())

# Evaluasi akurasi
accuracy_scratch = accuracy_score(y_val, y_pred_scratch)
print(f"Akurasi Gaussian Naive-Bayes from scratch: {accuracy_scratch:.2f}")


Akurasi Gaussian Naive-Bayes from scratch: 0.98


In [482]:


# Gaussian Naive-Bayes dengan scikit-learn
gnb_sklearn = GaussianNB()
gnb_sklearn.fit(X_train_resampled, y_train_resampled)
y_pred_sklearn = gnb_sklearn.predict(X_val)

# Evaluasi akurasi
accuracy_sklearn = accuracy_score(y_val, y_pred_sklearn)
print(f"Akurasi Gaussian Naive-Bayes (scikit-learn): {accuracy_sklearn:.2f}")


Akurasi Gaussian Naive-Bayes (scikit-learn): 0.84


In [483]:
# Kode bagian save CSV Error
# # Simpan hasil prediksi Gaussian Naive-Bayes dalam CSV
# # Prediksi data test
# y_pred_gnb = gnb_scratch.predict(test_data.drop(columns=['id']).to_numpy())
# # Simpan hasil prediksi Gaussian Naive-Bayes dalam CSV
# submission_gnb = test_data[['id']].copy()  # Kolom ID dari test_data
# submission_gnb['label'] = y_pred_gnb  # Tambahkan kolom prediksi
# submission_gnb.to_csv('submission_gnb_scratch.csv', index=False)
# print("Submission Gaussian Naive-Bayes from scratch disimpan sebagai 'submission_gnb_scratch.csv'")


## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [403]:
# Type your code here
import pickle

# Fungsi untuk menyimpan model
def save_model(model, filename):
    """
    Menyimpan model ke file .pkl.
    :param model: Model yang ingin disimpan.
    :param filename: Nama file untuk menyimpan model.
    """
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model disimpan ke {filename}")

# Fungsi untuk memuat model
def load_model(filename):
    """
    Memuat model dari file .pkl.
    :param filename: Nama file yang menyimpan model.
    :return: Model yang dimuat.
    """
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    print(f"Model dimuat dari {filename}")
    return model


In [404]:
# Simpan model KNN from scratch
save_model(knn_scratch, 'knn_scratch_model.pkl')

# Simpan model KNN scikit-learn
save_model(knn_sklearn, 'knn_sklearn_model.pkl')


Model disimpan ke knn_scratch_model.pkl
Model disimpan ke knn_sklearn_model.pkl


In [405]:
# Simpan model Gaussian Naive-Bayes from scratch
save_model(gnb_scratch, 'gnb_scratch_model.pkl')

# Simpan model Gaussian Naive-Bayes scikit-learn
save_model(gnb_sklearn, 'gnb_sklearn_model.pkl')


Model disimpan ke gnb_scratch_model.pkl
Model disimpan ke gnb_sklearn_model.pkl


In [ ]:
# Muat model KNN from scratch
loaded_knn_scratch = load_model('knn_scratch_model.pkl')

# Muat model KNN scikit-learn
loaded_knn_sklearn = load_model('knn_sklearn_model.pkl')


In [ ]:
# Muat model Gaussian Naive-Bayes from scratch
loaded_gnb_scratch = load_model('gnb_scratch_model.pkl')

# Muat model Gaussian Naive-Bayes scikit-learn
loaded_gnb_sklearn = load_model('gnb_sklearn_model.pkl')


# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`